Contents of the notebook:
1. Look at a few pythonic examples of how you can analyze and visualize data.
1. Simulate a few regularities concerning convergence of sample means. 
1. Make a few more experiments showing how different distributions arise.
1. Take look at the distribution of some financial data. 

You will have to just click throug these sections and play with them a little.

And after that, there will be the actual work to do: describe the distribution of two more financial variables. 


# 1. A few examples of what can you compute with data and distributions

**What can you do with random variables**:
* calculate their cdf, quantiles, moments, etc from data - or from formulas
* visualize data samples and populations
* sample data from distribution formulas
* fit distribution formulas to data samples

That's what we'll try here. We'l use the libraries `matplotlib`, `numpy`, `scipy` and `pandas`, which must be installed beforehand - e.g. using `pip` or `conda` package managers from command line. 

In [0]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline

## Calculating distribution properties

#### This is how distribution properties can be calculated from data

In [0]:
data = [1, 2, 3, 4, 9, 10, 11, 17, 30, 49, 121, 289, 503]

In [0]:
# moments
print(np.mean(data))
print(np.std(data))
print(scipy.stats.skew(data))
print(scipy.stats.kurtosis(data))

In [0]:
# linearly interpolated quantiles: min, 25% quantlie, median, 75% quantile, max
print(np.percentile(data, [0, 25, 50, 75, 100]))

#### And here these properties are calculated from population formulas

Scipy has a special family of objects that represent distributions and have lots of interesting methods. 

See more details in https://docs.scipy.org/doc/scipy/reference/stats.html

In [0]:
# formula for a normal with mean 90 and standard deviation 100
distribution = scipy.stats.norm(loc=90, scale=100)
print(type(distribution))

In [0]:
# analytical moments (someone took the integrals for you)
print(distribution.mean())
print(distribution.std())
print(distribution.stats('mvsk')) # mean, variance, skewness and kurtosis

In [0]:
# cdf at different points
print(distribution.cdf([10, 100, 200]))
# cdf at different points
print(distribution.pdf([10, 100, 200]))

In [0]:
# quantiles at different percentages
print(distribution.ppf([0.05, 0.5, 0.8, 0.95, 0.999]))

## Visualization

The most frequently used plot for distributions is histogram

In [0]:
plt.hist(data, bins=5);

Sometimes a histogram with unequal bins can be more informative - but don't forget to normalize it!

In [0]:
bins = [0, 10, 100, 500]
plt.subplot(1,2,1)
plt.hist(data, bins=bins);
plt.subplot(1,2,2)
plt.hist(data, bins=bins, density=True);

To visualize (something like) the CDF of a dataset, you can plot the variable again its cumulative probability in a dataset.

In [0]:
plt.plot(sorted(data), np.linspace(0, 1, num=len(data)))
plt.xlabel('values of the random variable')
plt.ylabel('values of its CDF')
plt.title('linearly interpolated empirical CDF');

You can plot a kernel density estimate (with normal, a.k.a. gaussian kernel).

In [0]:
grid = np.linspace(0, 550)
bandwidths = [None, 0.2, 1]
for bw in bandwidths:
    kde = scipy.stats.kde.gaussian_kde(data, bw_method=bw)
    plt.plot(grid, kde.evaluate(grid));
bandwidths[0] = 'automatic'
plt.legend(bandwidths)
plt.title('Kernel density estimates with different kernel width');

We can show how different methods of density estimation work with one more example.

In [0]:
np.random.seed(1)
plt.figure(figsize=(12, 4))
plt.subplot(1,3,1)

x = np.concatenate([
    np.random.normal(loc=1, scale=1, size=35), 
    np.random.normal(loc=3, scale=3, size=15)
])
plt.hist(x, bins=20, density=True);
grid = np.linspace(x.min() - 1, x.max() + 1, 1000)
plt.plot(
    grid,
    scipy.stats.norm.pdf(grid, loc=1, scale=1)*0.7 
    + scipy.stats.norm.pdf(grid, loc=3, scale=3)*0.3
)
plt.legend(['population density', 'histogram'])
plt.title('Histogram density estimation');

plt.subplot(1,3,2)
scales = [0.3,  3, 10]
for scale in scales:
    density = sum((np.abs(xi-grid) < scale * 0.5) / scale for xi in x) / len(x)
    plt.plot(grid, density);
plt.title('Moving window density estimate');
plt.legend(scales);

plt.subplot(1,3,3)
scales = [0.1,  1, 5]
for scale in scales:
    density = sum(scipy.stats.norm(xi, scale=scale).pdf(grid) for xi in x) / len(x)
    plt.plot(grid, density);
plt.title('Normal kernel density estimate');
plt.legend(scales);

And one more example of looking at log scale for PDF and CDF

In [0]:
np.random.seed(1)
pareto_x = scipy.stats.pareto.rvs(1.7, size=1000)
x_d = np.linspace(0, pareto_x.max() + 1, 1000)
density = scipy.stats.kde.gaussian_kde(pareto_x, bw_method=0.5).evaluate(x_d)

plt.figure(figsize=(10, 6))
plt.subplot(2,3,1)
plt.hist(pareto_x, bins=30, density=True)
plt.title('Histogram')

plt.subplot(2,3,2)
plt.plot(x_d, density)
plt.title('KDE')

plt.subplot(2,3,3)
plt.plot(x_d, density)
plt.yscale('log')
plt.title('KDE, log scale')

xs = np.array(sorted(pareto_x))
q = np.linspace(0, 1, num=len(pareto_x))

plt.subplot(2,3,4)
plt.plot(xs, q)
plt.title('CDF')

plt.subplot(2,3,5)
plt.plot(xs, 1-q)
plt.title('1 - CDF')

plt.subplot(2,3,6)
plt.plot(xs, 1-q)
plt.yscale('log')
plt.xscale('log')
plt.title('1 - CDF, log-log scale')

plt.tight_layout();

A brief way to plot several distribution is `boxplot` - it shows mean, 25% and 75% quantiles, range that contains most data ("whiskers"), and outliers.

In [0]:
data1 = [1, 2, 4, 8, 12, 19, 23, 3, 54, 32, 1, 39]
data2 = [3, 5, 32, 54, 8, 32, 2, 4, 42]

In [0]:
plt.boxplot([data1, data2], whis=[5, 95]);

A violinplot shows roughly the same, but it just more fancy. 

In [0]:
plt.violinplot([data1, data2]);

Bivariate data can be visualized as a scatterplot

In [0]:
pairs1 = [1, 2, 3, 4, 2.5, 3.5]
pairs2 = [1, 3, 6, 4, 4, 2]
plt.scatter(pairs1, pairs2);

Or you can plot a "genuine" (from a population, instead of a dataset) PDFs or CDFs.

In [0]:
x = np.linspace(-3, 5, num=1000)
parameters = [[0, 1], [2, 1], [0, 0.5], [1, 2]]
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
for mu, sigma in parameters:
    plt.plot(x, scipy.stats.norm(mu, sigma).pdf(x))
plt.legend(parameters)
plt.title('Normal PDFs with different parameters')
plt.subplot(1,2,2)
for mu, sigma in parameters:
    plt.plot(x, scipy.stats.norm(mu, sigma).cdf(x))
plt.legend(parameters)
plt.title('Normal CDFs with different parameters');

Probability plot, a.k.a. quantile-quantlie plot, is a great way to see how well some distribution family fits to a dataset.

If the fit is good, then most dots will lie close to the diagonal.

In [0]:
plt.figure(figsize=(8, 4))
plt.subplot(1,2,1)
scipy.stats.probplot(pareto_x, dist=scipy.stats.norm, sparams=(pareto_x.mean(), pareto_x.std()), plot=plt)
plt.title('Q-Q plot, normal')
plt.subplot(1,2,2)
scipy.stats.probplot(pareto_x, dist=scipy.stats.pareto, sparams=(pareto_x.mean(), pareto_x.std()), plot=plt)
plt.title('Q-Q plot, Pareto');
plt.tight_layout()

## The assignment : visualize the distribution of word frequencies

Let's calculate frequencies of all words from the previous homework. 

Your need to visualize the distribution of words frequencies (e.g. plot a histogram of them, or plot their CDF, in an appropriate scale). 

An additional question: from what family of probability distributions this data might have been generated?

In [0]:
%%capture
!wget https://raw.githubusercontent.com/avidale/ps4ds2019/master/homework/week1/spam_classifier/SMSSpamCollection

In [0]:
import re
import pandas as pd 
from collections import Counter
data = pd.read_csv('SMSSpamCollection', sep='\t', header=None)
data.columns = ['target', 'text']
words_frequencies = pd.Series(Counter(word for sent in data.text for word in re.split('\W+', sent.lower()) if word))

In [0]:
# todo: visualize the distribution of word frequencies 

# 2. Simulation of the Central Limit Theorem and Law of Large Numbers

The CLT states that sum of any 𝑛 (approximately) independent and (approximately) identical random variables will converge to normal distribution, as we increase 𝑛.

### How CLT looks like

You can take (almost) any distribution. The distribution may have any shape. 

In [0]:
np.random.seed(41)
samples = {'binomial':[], 'uniform':[], 'lognormal':[], 'exponential':[], 'pareto':[]}
ns = [1, 3, 10, 30, 100, 300, 1000, 3000]
for n in ns:
    samples['binomial'].append(np.random.binomial(n=1, p=0.5, size = (1000,n)).sum(axis = 1))
    samples['uniform'].append(np.random.uniform(size = (1000,n)).sum(axis = 1))
    samples['lognormal'].append(np.random.lognormal(0,1, size = (1000,n)).sum(axis = 1))
    samples['exponential'].append(np.random.exponential(0.1, size = (1000,n)).sum(axis = 1))
    samples['pareto'].append(np.random.pareto(3.5, size = (1000,n)).sum(axis = 1))
    
plt.figure(figsize=(12, 2))
for i, name in enumerate(samples):
    plt.subplot(1, 5, i+1)
    plt.hist(samples[name][0], bins = 20)
    plt.title(name)
plt.tight_layout()

But when you make a large sample from this distribution and calculate its average, it starts looking like normal. Independently of the original shape!

In [0]:
plt.figure(figsize=(20, 12))
for j, n in enumerate(ns):
    for i, name in enumerate(samples):
        plt.subplot(5, len(ns), i*len(ns)+1+j)
        plt.hist(samples[name][j], bins = 20)
        plt.title('{}, n={}'.format(name, n))
plt.tight_layout()

### When CLT works

In brief - when the underlying individual random variables have mean and variance themselves, then sampling sum converges to normal distribution.

As an example, we can sample longer and longer sums from the exponential distribution. 

Let's track how the different sample moments converge and/or diverge when we add lots of random variables together.

In [0]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline

def simulate_clt(distribution, size=100000, sample_sizes=(1, 2, 3, 10, 50, 100, 1000)):
    plt.figure(figsize=(14,2))
    np.random.seed(1)
    for i, n in enumerate(sample_sizes):
        sample_sum = distribution.rvs(size=(size, n)).sum(axis=1)
        plt.subplot(1, len(sample_sizes), i+1)
        plt.hist(sample_sum, bins=100);
        plt.yticks([])
        plt.title('n={}'.format(n))
        # TODO: calculate the first 4 moments for our random variable sample_sum
        # if you do it correctly, mean and variance in this example should diverge, and skewness and kurtosis - converge. 
        mean = np.mean(sample_sum)
        std_deviation = np.std(sample_sum)
        skewness = scipy.stats.skew(sample_sum)
        excess_kurtosis = scipy.stats.kurtosis(sample_sum)
        print('n={:4}, mean={:8.3f}, std.dev={:6.3f}, skewness={:6.3f}, excess kurtosis={:6.3f}'.format(
            n, mean, std_deviation, skewness, excess_kurtosis))
    plt.subplots_adjust(top=0.75)
    
distribution = scipy.stats.expon()
simulate_clt(distribution)
plt.suptitle('Distribution of sum of n exponential variables');

Another law of <s>nature</s> mathematics is the law of large numbers: it says that the mean of a large sample converges to the expected value of population from which the sample was taken.

Let's try to simulate it as well!

In [0]:
def plot_sequential_sample(distribution, size=(500, 10000)):
    np.random.seed(1)
    sequential_sample = np.cumsum(distribution.rvs(size=size), axis=1)
    sequential_sample /= np.arange(sequential_sample.shape[1]) + 1
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(sequential_sample.T, linewidth=0.2, color='k')
    plt.xlabel('sample size')
    plt.ylabel('sample mean')
    plt.subplot(1,2,2)
    plt.plot(sequential_sample.T, linewidth=0.2, color='k')
    plt.xlabel('sample size')
    plt.ylabel('sample mean')
    plt.xscale('log')
    if sequential_sample.min() > 0:
        plt.yscale('log')

Start with the same exponential distribution

In [0]:
plot_sequential_sample(scipy.stats.expon(1))

However, with more capricious Pareto distribution (with low values of the shape parameter) the law of large numbers seems to fail: instead of convergence, more and more outliers emerge. 

Moreover, instead of stabilizing, sample means grow (and with overwhelming speed!) when we increase sample size. 

In [0]:
plot_sequential_sample(scipy.stats.pareto(b=0.3))

The central limit theorem as well doesn't seem to work with this distribution. 

In [0]:
simulate_clt(scipy.stats.pareto(b=0.3))

## The assignment

For what parameters of Pareto distribution the Law of large numpers works, and for what parameters it does not?

Make a guess and try to demonstrate its correctness.

** todo ** : your guess here

In [0]:
# todo: your code here

# 3. A few more random processes

This section just shows you how different distributions can arise from simple random processes. 

### Reproduction of bacteria and lognormal distribution

Normal distribution emerges as a result of adding multiple similar variables. But sometimes in nature and in society we observe *multiplication* of random variables. In this case, resulting distribution may be lognormal. 

For example, let's consider a population of bacteria in a cup of broth. In favorable environments, they never die, and every bacterium every minute divides in two with probability 50%. In the first minute, there is only one bacterium. 

Q: What will be the distribution of number of bacteria in half an hour?

A: It seems that it will be lognormal, because on average the number of bacteria is multiplied by 1.5 on every minute. 

In [0]:
# let's write a function that generates bacteria!
def plot_germs(minutes=30, initial=1, p_divide=0.5):
    numbers_after_30_minutes = []
    # try it 10K times
    for i in range(10000):
        number_of_germs = initial
        for t in range(minutes):
            # all bacteria divide independently, so the number of successes (divisions) has binomial distribution
            number_of_germs += np.random.binomial(n=number_of_germs, p=p_divide)
        numbers_after_30_minutes.append(number_of_germs)
    h = plt.hist(numbers_after_30_minutes, bins=100, normed=True)
    return(h)
np.random.seed(42)
plot_germs(30);

In order to unserstand how the lognormal distribution emerges, let's look at the distribution after 2 minutes. 

In [0]:
plot_germs(2);

What are the probabilities of having 1, 2, 3 or 4 bacteria after 2 minutes ?

Yuo can find them analytically or just run the folliwing cell:

In [0]:
distr = {}
distr[1] = 0.5*0.5 # in both minutes there are no divisions
distr[2] = 0.5*0.5 + 0.5*0.5*0.5 # there is a division in the first minute and no divisions in the second, or vice versa
distr[3] = 0.5*0.5*0.5 + 0.5*0.5*0.5 # one division in the first minute, and one (of 2 possible) in the second minute
distr[4] = 0.5*0.5*0.5 # in the first minute the first bacterium divides, in the second - both
print(distr) 

If you still don't see it, let's print all possible scenarios as a tree.

In [0]:
def print_probabilities(t=0, t_max=2, n=1, p=1):
    """
    A recursive function that prints all possible reproduction scenarios
    """
    print('{}Minute {}: {} bacteria, probability {}'.format('\t'*t, t, n, p))
    if t < t_max:
        for n_new in range(0, n+1): # there can be from 0 to n new bacteria
            prob_n_new = scipy.stats.binom.pmf(n_new, n, 0.5)
            print_probabilities(t+1, t_max, n+n_new, p * prob_n_new)
print_probabilities(t_max = 2)

Pay attention to the probabilities on the last level (minute 2). 

### Lifecycle: the geometric/exponential distribution

One more question that we can ask about this model is the following: in what age do the bacteria usually divide? How is it distributed?


Let's consider a "newborn" bacterium. With probability $1/2$ it divides on it first minute. With probability $1/4$ it does not divide on the first minute, but divides on the second minute. With probability $1/8$ it divides on the 3rd minute, and so on... So probabilities decrease in geometric progression, which gives the name to the distribution. 

Let's try to generate it. 

In [0]:
# please run this cell for several times!
age = 0 # initially, the bacterium has age 0
while np.random.randint(2) == 0: # while a coin flips with 'tails', the bacterium does not divide ...
    age = age + 1 # ... but its age increases
print(age)

In [0]:
# let's repeat the experiment for 10K bacteria
ages = []
for t in range(10000):
    age = 0
    while np.random.randint(2) == 0: 
        age = age + 1
    ages.append(age)
plt.hist(ages, bins = max(ages)+1)
plt.title('Sample from distribution of bacteria lifetime');

## City sizes, internet memes, and Pareto law

Normal distribution is a result of adding independent variables, and it oftem emerges in systems with negative feedback or without feedback. But in society and economy there is often positive feedback. 

For example, the larger is the company budget, the more it can spend on advertizing, the more customers it has, the larger its budget... Or the more times a meme is reposted, the more eyes will see it, the more users will repost it... In general, the past reinforces the future, and this influence accumulates. 

Let' simulate the process of reposting the memes. Let's start from the situation of no reposts. 

In [0]:
# 1000 memes, each one posted only once. 
repost_counts = [1]
new_meme_probability = 0.01
plt.hist(repost_counts, bins=100);

Every new user chooses a meme randomly (with probability roughly proportional to the number of occurrences of this meme), and reposts it. 

In [0]:
# add new users. Everyone reposts a random post or makes a new one, and its number of occurrences increases by 1. 
for i in range(50000):
    if np.random.uniform() < new_meme_probability:
        repost_counts.append(1)
    else:
        proba = np.array(repost_counts) + 10.0 # an additional factor that makes chances more equal
        proba /=  sum(proba)
        chosen_post = np.random.choice(len(repost_counts), p=proba)
        repost_counts[chosen_post] += 1
plt.hist(repost_counts, bins=100);

After repeating the process multiple times, we see that there are a few 'leader' memes that leave all the competitors far behind. 

The plot of ($1-CDF$) (and of $PDF$, if we knew it) for Pareto distribution is close to linear in log-log scale

In [0]:
plt.plot(
    1 - np.linspace(0, 1, len(repost_counts)),
    sorted(repost_counts)
)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('x')
plt.ylabel('1-CDF');


The distributions of city sizes, personal incomes, and word occurences look pretty similar. 

# 4. Stock Prices data: example of modelling a distribution

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
%matplotlib inline

Here we download the data on the 500 top US public companies from a [website](https://datahub.io/core/s-and-p-500-companies-financials). 

In [0]:
url = 'https://datahub.io/core/s-and-p-500-companies-financials/r/constituents-financials.csv'
data = pd.read_csv(url)

Let's take a look at our data.

In [0]:
data.head()

### Example of analysis of a variable

Let's take stock price and see how it is distributed. 

Here we will use some of magical `pandas` methods. In the 3rd week, you'll study them in more details on the Python course. 

From the basic statistics of our variable, we can see that it is asymmetric: mean is larger then the median, skewness is positive. Excess kurtosis is very large, and it may be due just to a few outliers. 

In [0]:
print(data['Price'].describe())
print(scipy.stats.skew(data['Price']))
print(scipy.stats.kurtosis(data['Price']))

I make a histogram of the data, but because of outliers it doesn't show me in much details its most common part. 

In [0]:
data['Price'].hist(bins=50, density=True);

Let's zoom in by plotting a histogram conditional on price being not too large.

It still looks asymmetric. 

In [0]:
data['Price'][data['Price'] <= 500].hist(bins=50, density=True);

What kind of distribution function might describe this data? Maybe, Poisson? It's also positive and asymmetric.

Let's generate a sample from Poisson distribution and compare its properties with the properties of our data.

In [0]:
x = pd.Series(np.random.poisson(data['Price'].mean(), size=500))
print(x.describe())
print(scipy.stats.skew(x))
print(scipy.stats.kurtosis(x))

Well, most of them don't coincide with the properties of our data. Median is too high; variance, skewness and kurtosis are too small.

And if we plot the histoframs of our distributions, we'll see that they are very different. 

In [0]:
x.hist(density=True)
data['Price'].hist(density=True, bins=50, alpha=0.5)
plt.legend(['poisson', 'data'])
plt.title('histogram of Poisson distribution vs histogram of data');

Maybe, our distributions is then lognormal?

We could check this hypothesis by first plotting the histogram for the logarighm of our variable.

In [0]:
log_price = np.log(data['Price'])
log_price.hist(bins=30, density=True);

Well, at least it's symmetric. 

And it's quantiles are quite close to the quantiles of normal distribution with the same location and scale.

In [0]:
log_price.describe()

In [0]:
z = pd.Series(np.random.normal(loc=log_price.mean(), scale=log_price.std(), size=500))
z.describe()

Histograms of the two non-logarithmized distributions look similarly as well. 

In [0]:
np.exp(z).hist(bins=30, density=True)
data['Price'][data['Price'] <= 1000].hist(bins=30, alpha=0.5, density=True);

Finally, let's make a q-q plot to compare all the quantiles.

In [0]:
scipy.stats.probplot(
    data['Price'],
    dist=scipy.stats.lognorm(s=log_price.std(), scale=np.exp(log_price.mean()), loc=1),
    plot=plt
);

We see that all the quantiles up to 400 align with lognormal quantiles almost perfectly, but after that, a different trend is observed. 

It may mean that we in fact work with a mixture of two different distributions!

We can think than the largest ~10 points are outliers and just ignore them. 

Or we can assume we have two different log-normal distributions, and fit them to our data (in a naive way).  

In [0]:
threshold = 350

part1 = scipy.stats.lognorm(
    s=log_price[data['Price'] < threshold].std(), 
    scale=np.exp(log_price[data['Price'] < threshold].mean())
)
part2 = scipy.stats.lognorm(
    s=log_price[data['Price'] >= threshold].std(), 
    scale=np.exp(log_price[data['Price'] >= threshold].mean())
)
proportion = data['Price'] >= threshold

For a mixture, there is no straigthforward way to calculate quantiles, so instead we'll just sample from it several times, and see whether these samples on average resemble the expected values.

In [0]:
np.random.seed(1)
for i in range(30):
    sample = sorted(np.concatenate([
        part1.rvs(size=sum(data['Price'] < threshold)),
        part2.rvs(size=sum(data['Price'] >= threshold))
    ]))
    plt.scatter(sample, sorted(data['Price']), s=2, color='b')

plt.plot(plt.ylim(), plt.ylim(), color='r');

Well, on average our sampled distribution almost does align with the data (however, most of the plot lies dangerously below the diagonal). 

One more good idea is to look at the same plot in log scale. 

In [0]:
np.random.seed(1)
for i in range(30):
    sample = sorted(np.concatenate([
        part1.rvs(size=sum(data['Price'] < threshold)),
        part2.rvs(size=sum(data['Price'] >= threshold))
    ]))
    plt.scatter(sample, sorted(data['Price']), s=2, color='b')
    
plt.xscale('log')
plt.yscale('log')

plt.plot(plt.ylim(), plt.ylim(), color='r');

Here it looks more okay. There is a strange group of poits at the bottom - but it corresponts to only one original observation.

In [0]:
data['Price'].sort_values().head()

By the way, stock prices seem to depend a lot on the sector. We can visualize it, but we won't go deeper here. 

In [0]:
data.boxplot(column='Price', by='Sector', figsize=(6, 6), vert=False);
plt.xscale('log');

# 5. The assignment

Take two other variables from the same dataset, `EBITDA` and `Market Cap`. For each of them, 
* calculate descriptive statsistics and comment on them;
* try different ways of visualizing the variable;
* try to fit a 'theoretical' distribution for this variable. 

There is no single "right" way to do this assignment. 

But the more interesting observations you will make about the two variables, the better it would be.